On the next stage I need to download file with ortologus rows in the system. I have mounted google drive with project files (in the future I'm going to use GitHub for these purposes). 

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

### Orto rows with singletons, but without paralog-containing rows

In [3]:
# Uploading and modificating orto rows tables
orto_rows = pd.read_csv("../data/C_psittaci.proteinortho.tsv", sep="\t") # Uploading dataframe with orto rows
orto_rows = orto_rows.rename(columns = {"# Species": "Strains"})
orto_rows["ortologus_row"] = orto_rows.index + 1 # Creating ortorows numbers

# Uploading first csv-table and creating a new column in it
orto_rows_list = orto_rows.index
C_psittaci_df1 = pd.read_csv("../data/First_table.csv")
C_psittaci_df1["ortologus_row"] = 0

#Filling orto_row column (sounds like an oxymoron)
organism = "Chlamydia psittaci"
for strain in tqdm(set(C_psittaci_df1["p_c_unity"])):
    maskstring = f"C_psittaci{strain}.fasta"   
    orrow = orto_rows.loc[:, maskstring].str[:14]
    for index in orto_rows_list:
        C_psittaci_df1.loc[C_psittaci_df1[C_psittaci_df1["id"] == orrow[index]].index, "ortologus_row"] = index + 1

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [00:44<00:00,  1.66s/it]


In [4]:
# Number of the paralogs
print("Number of the paralogs =", sum(orto_rows.query("Genes > Strains").Genes) - sum(orto_rows.query("Genes > Strains").Strains))

Number of the paralogs = 65


In [5]:
# Creating a subset without pararows
pararows_numbers = orto_rows.query("Genes > Strains").ortologus_row
C_psittaci_df1 = C_psittaci_df1.query("ortologus_row not in @pararows_numbers").query("ortologus_row != 0")

In [7]:
# Compiling data about start-codons of ortologus rows
start_codon_per_row = C_psittaci_df1.groupby("ortologus_row", as_index=False).agg({"start_codone": ".".join})
start_codon_per_row["start_codone"]
orr_start_list = []
for number in tqdm(range(len(start_codon_per_row))):
    orr_start_list.append(start_codon_per_row.iloc[number, 1].split("."))
start_codons = pd.DataFrame(
    {
        "ortologus_row": start_codon_per_row["ortologus_row"],
        "start_codons": pd.Series(orr_start_list),
        "ATG": 0.0,
        "GTG": 0.0,
        "TTG": 0.0,
    }
)

# Computing frequencies of exact start-codons
start_codons.sort_values("ortologus_row", inplace=True)
for row in tqdm(range(len(start_codons))):
    freqs = pd.Series(start_codons["start_codons"][row]).value_counts() # If it will be need to visualize percents, use normalize=True in brackets
    rowlength = len(start_codons["start_codons"][row])
    if "ATG" in freqs:
        start_codons["ATG"][row] = freqs["ATG"]
    else:
        start_codons["ATG"][row] = 0
    if "GTG" in freqs:
        start_codons["GTG"][row] = freqs["GTG"]
    else:
        start_codons["GTG"][row] = 0
    if "TTG" in freqs:
        start_codons["TTG"][row] = freqs["TTG"]
    else:
        start_codons["TTG"][row] = 0

  0%|                                                                                                               | 0/1525 [00:00<?, ?it/s]/tmp/ipykernel_5007/1178007494.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["ATG"][row] = freqs["ATG"]
/tmp/ipykernel_5007/1178007494.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["GTG"][row] = 0
/tmp/ipykernel_5007/1178007494.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

  9%|█████████▎                                                                                         | 144/1525 [00:00<00:00, 1432.20it/s]/tmp/ipykernel_5007/1178007494.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["ATG"][row] = freqs["ATG"]
/tmp/ipykernel_5007/1178007494.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["GTG"][row] = 0
/tmp/ipykernel_5007/1178007494.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

 19%|██████████████████▋                                                                                | 288/1525 [00:00<00:00, 1394.16it/s]/tmp/ipykernel_5007/1178007494.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["ATG"][row] = freqs["ATG"]
/tmp/ipykernel_5007/1178007494.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["GTG"][row] = 0
/tmp/ipykernel_5007/1178007494.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

 28%|███████████████████████████▊                                                                       | 428/1525 [00:00<00:00, 1233.29it/s]/tmp/ipykernel_5007/1178007494.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["ATG"][row] = freqs["ATG"]
/tmp/ipykernel_5007/1178007494.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["GTG"][row] = 0
/tmp/ipykernel_5007/1178007494.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

 37%|████████████████████████████████████▍                                                              | 562/1525 [00:00<00:00, 1271.26it/s]/tmp/ipykernel_5007/1178007494.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["ATG"][row] = freqs["ATG"]
/tmp/ipykernel_5007/1178007494.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["GTG"][row] = 0
/tmp/ipykernel_5007/1178007494.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

...

 81%|███████████████████████████████████████████████████████████████████████████████▊                  | 1242/1525 [00:00<00:00, 1297.16it/s]/tmp/ipykernel_5007/1178007494.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["ATG"][row] = freqs["ATG"]
/tmp/ipykernel_5007/1178007494.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["GTG"][row] = 0
/tmp/ipykernel_5007/1178007494.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

/tmp/ipykernel_5007/1178007494.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["TTG"][row] = 0
/tmp/ipykernel_5007/1178007494.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["ATG"][row] = freqs["ATG"]
/tmp/ipykernel_5007/1178007494.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["GTG"][row] = 0
/tmp/ipykernel_5007/1178007494.py:33: SettingWithCopyWarning: 
A value is trying to be se

 90%|████████████████████████████████████████████████████████████████████████████████████████▏         | 1373/1525 [00:01<00:00, 1299.61it/s]/tmp/ipykernel_5007/1178007494.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["ATG"][row] = freqs["ATG"]
/tmp/ipykernel_5007/1178007494.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["GTG"][row] = 0
/tmp/ipykernel_5007/1178007494.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

 99%|█████████████████████████████████████████████████████████████████████████████████████████████████▎| 1514/1525 [00:01<00:00, 1332.02it/s]/tmp/ipykernel_5007/1178007494.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["ATG"][row] = freqs["ATG"]
/tmp/ipykernel_5007/1178007494.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["GTG"][row] = 0
/tmp/ipykernel_5007/1178007494.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 1525/1525 [00:01<00:00, 1309.88it/s]


In [9]:
# Computing uniformity of start-codon per ortologus row
start_codons["uniformity"] = "NA"
for row in range(len(start_codons)):
    if len(set(start_codons.iloc[row, 1])) == 1:
        start_codons.iloc[row, 5] = "uniform"
    else:
        start_codons.iloc[row, 5] = "non-uniform"

In [11]:
# Constructing table withh all data about the ortologus rows (including pararows)
start_codons2 = start_codons.merge(orto_rows, on="ortologus_row", how="outer")

In [13]:
# Creating a table, combining data about gene and its start-codone
strain_gene_row = start_codons2[["Strains", "Genes", "ortologus_row", "uniformity", "ATG", "GTG", "TTG"]]   # Other codons deleted
summary_rows = C_psittaci_df1.merge(strain_gene_row, on="ortologus_row")

In [15]:
# Identifying non-uniform rows


We need to align only rows with different start-codons.

In [ ]:
non_uniform_or_list = list(set(start_codons2.query("uniformity == 'non-uniform'").ortologus_row))
for orto_row in tqdm(non_uniform_or_list):
    subset = summary_rows.query("ortologus_row == @orto_row")
    with open ("../data/multialignments/C_psittaci_withoutp_" + str(orto_row) + ".fasta", "w") as nucleotide_fasta:
        for index, row in subset.iterrows():
            if row['type_of_the_gene'] != "pseudogene":
                nucleotide_fasta.write(">")
                nucleotide_fasta.write(row["id"])
                nucleotide_fasta.write("_")
                nucleotide_fasta.write(row["source"])
                nucleotide_fasta.write("_")
                nucleotide_fasta.write(row["start_codone"])
                nucleotide_fasta.write("\n")
                nucleotide_fasta.write(row["n_sequence"])
                nucleotide_fasta.write("\n")

#### Saving summary-rows and start-codons2

In [ ]:
summary_rows.to_csv("../data/summary_rows_prokka.csv", index=False)

In [ ]:
start_codons2.to_csv("../data/start_codons2_prokka.csv", index=False)